In [1]:
import os
import openai

from langchain_openai.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain.memory import ConversationBufferWindowMemory

In [2]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

llm_name = "gpt-3.5-turbo"

In [3]:
def load_db(file, chain_type, k):
    # load transcript
    transcript = TextLoader(file).load()
    # split documents
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=70)
    docs = text_splitter.split_documents(transcript)
    # define embedding
    embeddings = OpenAIEmbeddings()                                                     # will be different for Cohere AI
    # create vector database from data
    db = Chroma.from_documents(docs, embeddings)
    # define retriever
    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": k})
    # retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

    # create a chatbot chain. Memory is managed externally.
    qa = ConversationalRetrievalChain.from_llm(
        llm = ChatOpenAI(temperature=0),                      #### Prompt Template is yet to be created
        chain_type=chain_type,                                #### Refine 
        retriever=retriever, 
        return_source_documents=True,
        return_generated_question=True,
        # memory=memory
    )
    
    return qa 

In [4]:
def buffer(history, buff):
    # basically a filter function
    # takes the chat history, ensures only buff latest elements exist
    if len(history) > buff :
        print(len(history)>buff)
        return history[-buff:]
    else:
        return history

In [5]:
chat_history = []

In [6]:
qa = load_db("./sample.txt", 'stuff', 4)
query = "Namaste! I need your help"
result = qa({'question': query, 'chat_history': chat_history}) #######################################

/home/maxx/.conda/envs/vcdeploy/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [7]:
type(qa)

langchain.chains.conversational_retrieval.base.ConversationalRetrievalChain

In [7]:
answer = result['answer']
answer

"Namaste! I'm here to help. What do you need assistance with?"

In [8]:
result

{'question': 'Namaste! I need your help',
 'chat_history': [],
 'answer': "Namaste! I'm here to help. What do you need assistance with?",
 'source_documents': [Document(page_content='~160~Aurora nodded and waved her magic wand, casting a spell that spread throughout the forest. From that day on, the animals frolicked and played in harmony, their hearts filled with joy and gratitude.\n~180~As Sammy climbed down from the Golden Acorn Tree, he felt a warm glow of happiness inside. He may have only found one golden acorn, but he had discovered something even more precious – the power of kindness and the magic of friendship.', metadata={'source': './sample.txt'}),
  Document(page_content='~40~One sunny morning, as Sammy scampered through the trees, he stumbled upon a mysterious path he had never seen before. Intrigued, he decided to follow it. The path led him deeper and deeper into the forest until he reached a clearing filled with colorful flowers and fluttering butterflies.', metadata={'

In [19]:
chat_history.extend([(query, result['answer'])])
chat_history = buffer(chat_history,3)

In [20]:
query = "What is the moral of the story?"
result = qa({"question": query, "chat_history": chat_history}) #######################################

In [21]:
result

{'question': 'What is the moral of the story?',
 'chat_history': [('Namaste! I need your help',
   "Namaste! I'm here to help. What do you need assistance with?")],
 'answer': 'The moral of the story is that kindness, friendship, courage, and love can lead to magical adventures and wonderful experiences.',
 'source_documents': [Document(page_content='~160~Aurora nodded and waved her magic wand, casting a spell that spread throughout the forest. From that day on, the animals frolicked and played in harmony, their hearts filled with joy and gratitude.\n~180~As Sammy climbed down from the Golden Acorn Tree, he felt a warm glow of happiness inside. He may have only found one golden acorn, but he had discovered something even more precious – the power of kindness and the magic of friendship.', metadata={'source': './sample.txt'}),
  Document(page_content='~200~And so, with a twinkle in his eye and a skip in his step, Sammy the squirrel continued his adventures in the enchanted forest, knowi

In [22]:
answer = result['answer']
answer

'The moral of the story is that kindness, friendship, courage, and love can lead to magical adventures and wonderful experiences.'

In [23]:
chat_history.extend([(query,result['answer'])])
chat_history = buffer(chat_history,3)

In [24]:
query = "Can you give a relevant title to the story?"
result = qa({"question": query, "chat_history": chat_history}) #######################################

In [25]:
result

{'question': 'Can you give a relevant title to the story?',
 'chat_history': [('Namaste! I need your help',
   "Namaste! I'm here to help. What do you need assistance with?"),
  ('What is the moral of the story?',
   'The moral of the story is that kindness, friendship, courage, and love can lead to magical adventures and wonderful experiences.')],
 'answer': 'A relevant title for the story could be "Sammy\'s Enchanted Forest Adventure."',
 'source_documents': [Document(page_content='~40~One sunny morning, as Sammy scampered through the trees, he stumbled upon a mysterious path he had never seen before. Intrigued, he decided to follow it. The path led him deeper and deeper into the forest until he reached a clearing filled with colorful flowers and fluttering butterflies.', metadata={'source': './sample.txt'}),
  Document(page_content='~160~Aurora nodded and waved her magic wand, casting a spell that spread throughout the forest. From that day on, the animals frolicked and played in ha

In [26]:
answer = result['answer']
answer

'A relevant title for the story could be "Sammy\'s Enchanted Forest Adventure."'

In [27]:
chat_history.extend([(query,result['answer'])])
chat_history = buffer(chat_history,3)

In [28]:
query = "which timestamp resembles moral of the story?"
result = qa({"question": query, "chat_history": chat_history}) #######################################

In [29]:
answer = result['answer']
answer

'The timestamp ~180~ where Sammy discovers the power of kindness and the magic of friendship best represents the moral of the story.'

In [30]:
chat_history.extend([(query,result['answer'])])
chat_history = buffer(chat_history,3)

True


In [31]:
chat_history

[('What is the moral of the story?',
  'The moral of the story is that kindness, friendship, courage, and love can lead to magical adventures and wonderful experiences.'),
 ('Can you give a relevant title to the story?',
  'A relevant title for the story could be "Sammy\'s Enchanted Forest Adventure."'),
 ('which timestamp resembles moral of the story?',
  'The timestamp ~180~ where Sammy discovers the power of kindness and the magic of friendship best represents the moral of the story.')]

---

**The Defalut Buffer Memory that store entire chat and provides it as context to the next conversation** ⬆

- Memory : Conv Buffer Window Memory (k=6) / Conv Summ Buff Memory (max_token_limit=650)
- Chain Type : Refine
- Prompt : Template Fine-Tuning
- Deployment : Gunicorn / Streamlit ? 

References: 
https://python.langchain.com/docs/use_cases/question_answering/

https://python.langchain.com/docs/modules/memory/types/token_buffer

In [15]:
dict(result['source_documents'][1])['page_content']

'~40~One sunny morning, as Sammy scampered through the trees, he stumbled upon a mysterious path he had never seen before. Intrigued, he decided to follow it. The path led him deeper and deeper into the forest until he reached a clearing filled with colorful flowers and fluttering butterflies.'

In [34]:
import re
pattern = r'~(\d+)~'

In [38]:
backlinked_docs = [result['source_documents'][i].page_content for i in range(len(result['source_documents']))]
timestamps = list(map(lambda s: int(re.search(pattern, s).group(1)) if re.search(pattern, s) else None, backlinked_docs))


In [39]:
dict(timestamps=timestamps, answer=result['answer'])

{'timestamps': [160, 40, 120, 10],
 'answer': "Namaste! I'm here to help. What do you need assistance with?"}